In [1]:
# !pip install -U bitsandbytes
# !pip install trl peft accelerate transformers

  Using cached trl-0.19.0-py3-none-any.whl.metadata (10 kB)
  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.19.0-py3-none-any.whl (375 kB)
Using cached datasets-3.6.0-py3-none-any.whl (491 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType
import torch
import numpy as np
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import bitsandbytes as bnb
from trl import SFTTrainer
import random
from datasets import Dataset

from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments)
import os

In [ ]:
path = '..'
os.chdir(path)
print(os.getcwd())

In [3]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
def initiate_base_model(base_model_name):
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype="bfloat16",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )

    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        device_map="auto",
        torch_dtype="float16",
        quantization_config=bnb_config
    )

    model.config.use_cache = False

    tokenizer = AutoTokenizer.from_pretrained(base_model_name)

    tokenizer.pad_token_id = tokenizer.eos_token_id

    return model, tokenizer

In [6]:
model, tokenizer = initiate_base_model(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [7]:
lora_config = LoraConfig(
    r=8,                     # Rank da matriz de atualização LoRA. Valores típicos: 8, 16, 32, 64.
    lora_alpha=16,           # Parâmetro de escalonamento Alpha. Geralmente r*2 ou r.
    target_modules=[         # Módulos onde os adaptadores LoRA serão aplicados.
        "q_proj",            # Varia conforme o modelo. Comum para transformers:
        "o_proj",            # "q_proj", "k_proj", "v_proj", "o_proj" (atenção)
        "k_proj",            # "gate_proj", "up_proj", "down_proj" (MLP/FeedForward)
        "v_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,       # Dropout para as camadas LoRA.
    bias="none",             # Se bias deve ser treinado ("none", "all", "lora_only"). "none" é comum.
    task_type="CAUSAL_LM",   # Tipo de tarefa.
)

In [8]:
model = get_peft_model(model, lora_config)

In [9]:
def format_prompt(prompt):
    system_msg = prompt[0]
    user_msg, expected_output = prompt[1].split('SQL Answer: ')

    return {
        "text": f"<s>[INST] <<SYS>>\n{system_msg}\n<</SYS>>\n\n{user_msg}\n[/INST] {expected_output}</s>"
    }

In [10]:
def tokenize(example):
    return tokenizer(example["text"])

In [12]:
seed = 42
training_data_percentage_usage = 0.8
np.random.seed(seed)

prompts = np.load('training_prompts.npy')
np.random.shuffle(prompts)
training_prompts = prompts[:int( (len(prompts) * training_data_percentage_usage) )]
validation_prompts = prompts[int( (len(prompts) * training_data_percentage_usage) ):]

training_prompts = list(map(format_prompt, training_prompts))
validation_prompts = list(map(format_prompt, validation_prompts))

tokenized_dataset = Dataset.from_list(list(map(tokenize, training_prompts)))
val_tokenized_dataset = Dataset.from_list(list(map(tokenize, validation_prompts)))

In [13]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [14]:
training_arguments = TrainingArguments(
    output_dir='fine_tunned_model',       # Diretório para salvar os resultados
    num_train_epochs=1,                   # Número de épocas (poucas para este lab devido ao tempo/recursos)
                                          # Aumente para 3-5 para melhores resultados se o dataset for maior.
    per_device_train_batch_size=1,        # Batch size por GPU. Aumente se a memória permitir (ex: 2, 4).
    gradient_accumulation_steps=4,        # Acumula gradientes para simular um batch size maior. Batch_efetivo = batch_size * num_gpus * accum_steps
                                          # Se der erro, comente e SFTTrainer usará o padrão (adamw).
    optim="adamw_torch",                  # Alternativa mais padrão se paged_adamw_8bit causar problemas.
    save_steps=100,                       # Salvar checkpoint a cada N passos.
    logging_steps=50,                     # Registrar logs a cada N passos.
    # learning_rate=1e-4,                   # Taxa de aprendizado. Comum para QLoRA.
    learning_rate=5e-5,                   # Taxa de aprendizado. Comum para QLoRA.
    weight_decay=0.001,                   # Decaimento de peso.
    fp16=False,                            # Usar precisão mista (float16). Requer GPU com suporte.
    bf16=True if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else False, # Usar bfloat16 se suportado, melhor que fp16
    max_grad_norm=0.3,                    # Clipping de gradiente.
    max_steps=-1,                         # Número máximo de passos de treino. -1 = usa num_train_epochs.
    warmup_ratio=0.03,                    # Proporção de passos para aquecimento do learning rate.
    group_by_length=True,                 # Agrupar amostras por tamanho para eficiência (bom para SFT).
    lr_scheduler_type="cosine",           # Tipo de scheduler do learning rate.
    report_to="none",
    eval_strategy="steps",
    eval_steps = 0.2
)

In [15]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenized_dataset,
    eval_dataset=val_tokenized_dataset,
    data_collator=data_collator  # ✅ No tokenizer here
)

Truncating train dataset:   0%|          | 0/6648 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1662 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [16]:
trainer.train()

Step,Training Loss
50,0.859200
100,0.010000
150,0.000100
200,0.000000
250,0.000000
300,0.000000
350,0.000000
400,0.000000
450,0.000000
500,0.000000


TrainOutput(global_step=1662, training_loss=0.02615820344541708, metrics={'train_runtime': 12804.371, 'train_samples_per_second': 0.519, 'train_steps_per_second': 0.13, 'total_flos': 3.073975780418519e+17, 'train_loss': 0.02615820344541708})

In [17]:
model.save_pretrained("./llama3-8b-qlora-adapter")
tokenizer.save_pretrained("./llama3-8b-qlora-adapter")

('./llama3-8b-qlora-adapter/tokenizer_config.json',
 './llama3-8b-qlora-adapter/special_tokens_map.json',
 './llama3-8b-qlora-adapter/chat_template.jinja',
 './llama3-8b-qlora-adapter/tokenizer.json')

In [19]:
!zip -r llama3-8b-qlora-adapter-LR5e5.zip llama3-8b-qlora-adapter

updating: llama3-8b-qlora-adapter/ (stored 0%)
updating: llama3-8b-qlora-adapter/special_tokens_map.json (deflated 63%)
updating: llama3-8b-qlora-adapter/tokenizer_config.json (deflated 96%)
updating: llama3-8b-qlora-adapter/tokenizer.json (deflated 85%)
updating: llama3-8b-qlora-adapter/adapter_config.json (deflated 56%)
updating: llama3-8b-qlora-adapter/README.md (deflated 66%)
updating: llama3-8b-qlora-adapter/adapter_model.safetensors (deflated 8%)
updating: llama3-8b-qlora-adapter/chat_template.jinja (deflated 52%)
